In [1]:
# Set arguments
PrimaryKeys = "HashedPKColumn"
IsIncremental = False

SourceWorkspace= ""
SourceLakehouse =""
SourceLakehouseName =''
source_schema = ""
source_name = ""

TargetWorkspace= ""
TargetLakehouse =""
TargetLakehouseName =''
target_schema = ""
target_name = ""


StatementMeta(, 6b0e549f-d44f-4d95-a0d3-c56f16f829b0, 3, Finished, Available)

## Load Libraries

In [2]:
import re
import datetime
import json
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *
from notebookutils import mssparkutils
import uuid

StatementMeta(, 6b0e549f-d44f-4d95-a0d3-c56f16f829b0, 4, Finished, Available)

## Define Starttime

In [3]:
start_audit_time = datetime.datetime.now()

StatementMeta(, 6b0e549f-d44f-4d95-a0d3-c56f16f829b0, 5, Finished, Available)

## Set Configuration

In [4]:
#Make sure you have enabled V-Order

spark.conf.set("sprk.sql.parquet.vorder.enabled", "true")

StatementMeta(, 6b0e549f-d44f-4d95-a0d3-c56f16f829b0, 6, Finished, Available)

## Set your loading paths

In [5]:
#Set SourceFile and target Location
source_changes_data_path = f"abfss://{SourceWorkspace}@onelake.dfs.fabric.microsoft.com/{SourceLakehouse}/Tables/{source_schema}{source_name}"
print(source_changes_data_path)

#Beware 
target_data_path = f"abfss://{TargetWorkspace}@onelake.dfs.fabric.microsoft.com/{TargetLakehouse}/Tables/{target_schema}{target_name}"
print(target_data_path)


StatementMeta(, 6b0e549f-d44f-4d95-a0d3-c56f16f829b0, 7, Finished, Available)

abfss://3a561f99-edc7-4d6c-a866-f3bf70bc7235@onelake.dfs.fabric.microsoft.com/1e10a8ff-69b8-46d5-8cb5-5fb5a3c88bbc/Tables/ApplicationPeople
abfss://3a561f99-edc7-4d6c-a866-f3bf70bc7235@onelake.dfs.fabric.microsoft.com/a81d105e-5c8b-44d7-922b-b2fe549490e0/Tables/ApplicationPeople


## Load from Bronze

In [6]:
#Read all incoming changes in Parquet format
dfDataChanged= spark.read\
                .format("delta") \
                .load(f"{source_changes_data_path}")

StatementMeta(, 6b0e549f-d44f-4d95-a0d3-c56f16f829b0, 8, Finished, Available)

## Add Audit Columns

In [7]:
dfDataChanged = dfDataChanged.withColumn('IsCurrent', lit(True).cast('boolean'))
dfDataChanged = dfDataChanged.withColumn('RecordStartDate', current_timestamp())
dfDataChanged = dfDataChanged.withColumn('RecordModifiedDate', current_timestamp())
dfDataChanged = dfDataChanged.withColumn('RecordEndDate', lit('9999-12-31').cast('timestamp'))
dfDataChanged = dfDataChanged.withColumn('IsDeleted', lit(False).cast('boolean'))

StatementMeta(, 6b0e549f-d44f-4d95-a0d3-c56f16f829b0, 9, Finished, Available)

In [8]:
#display(dfDataChanged)

StatementMeta(, 6b0e549f-d44f-4d95-a0d3-c56f16f829b0, 10, Finished, Available)

## Read Original if exists

In [9]:
#Check if Target exist, if exists read the original data if not create table and exit
if DeltaTable.isDeltaTable(spark, target_data_path):
    # Read original/current data
    dfDataOriginal = (spark
                        .read.format("delta")
                        .load(target_data_path)
                        )

else:
    # Use first load when no data exists yet and then exit 
    dfDataChanged.write.format("delta").mode("overwrite").save(target_data_path)
    TotalRuntime = str((datetime.datetime.now() - start_audit_time)) 

    # Your data
    result_data = {
        "CopyOutput":{
            "Total Runtime": TotalRuntime,
            "TargetSchema": target_schema,
            "TargetName" : target_name
        }
        }


    mssparkutils.notebook.exit(result_data)

StatementMeta(, 6b0e549f-d44f-4d95-a0d3-c56f16f829b0, 11, Finished, Available)

## Add columns for Merge SCD 2

In [10]:
#add a new column MergeKey based on the HashedPKColumn
dfDataChanged = dfDataChanged.withColumn('MergeKey', dfDataChanged['HashedPKColumn'])
dfDataChanged = dfDataChanged.withColumn('Action', lit('U'))

StatementMeta(, 6b0e549f-d44f-4d95-a0d3-c56f16f829b0, 12, Finished, Available)

## Check for changes

In [11]:
# Define columns to insert and delete
columns_to_insert_deletes = {f"changes.{column}" for column in dfDataOriginal.columns if column not in ('HashedPKColumn', 'MergeKey', 'Action')}

# Join DataFrames for inserts and deletes
df_inserts_deletes = dfDataChanged.alias('changes').join(dfDataOriginal.alias('original'),
                                                    (dfDataChanged.HashedPKColumn == dfDataOriginal.HashedPKColumn) &
                                                    (dfDataOriginal.IsCurrent == 1) &
                                                    (dfDataOriginal.IsDeleted == 0),
                                                    how='full') \
    .where("""(original.IsCurrent== 1 AND original.IsDeleted== 0
                    AND original.HashedNonKeyColumns <> changes.HashedNonKeyColumns) 
                OR original.HashedPKColumn is null
                OR changes.MergeKey is null""") \
    .select(
    "original.HashedPKColumn", 
    expr("case when original.IsCurrent is null or original.HashedNonKeyColumns <> changes.HashedNonKeyColumns then 'I' when changes.MergeKey is null then 'D' end as Action"),
    expr("case when original.IsCurrent is null or original.HashedNonKeyColumns <> changes.HashedNonKeyColumns then NULL when changes.MergeKey is null then original.HashedPKColumn else changes.MergeKey end as MergeKey"),
    *columns_to_insert_deletes
    )

# Create a final staging table as src for merge
dfDataChanged = df_inserts_deletes.unionByName(dfDataChanged)
dfDataChanged = dfDataChanged.withColumn('RecordEndDate',
                                        expr("""case when Action='U' 
        then RecordStartDate - interval 0.001 seconds
        else RecordEndDate end"""))

StatementMeta(, 6b0e549f-d44f-4d95-a0d3-c56f16f829b0, 13, Finished, Available)

In [12]:
#display(dfDataChanged)

StatementMeta(, 6b0e549f-d44f-4d95-a0d3-c56f16f829b0, 14, Finished, Available)

## Merge table

In [13]:
columns_to_insert = {column: f"updates.{column}" for column in dfDataOriginal.columns if column not in ('IsCurrent', 'HashedNonKeyColumns', 'RecordStartDate', 'HashedPKColumn', 'RecordModifiedDate', 'RecordEndDate', 'IsDeleted', 'MergeKey', 'Action')}
print(columns_to_insert)

StatementMeta(, 6b0e549f-d44f-4d95-a0d3-c56f16f829b0, 15, Finished, Available)

{'PersonID': 'updates.PersonID', 'FullName': 'updates.FullName', 'PreferredName': 'updates.PreferredName', 'SearchName': 'updates.SearchName', 'IsPermittedToLogon': 'updates.IsPermittedToLogon', 'LogonName': 'updates.LogonName', 'IsExternalLogonProvider': 'updates.IsExternalLogonProvider', 'HashedPassword': 'updates.HashedPassword', 'IsSystemUser': 'updates.IsSystemUser', 'IsEmployee': 'updates.IsEmployee', 'IsSalesperson': 'updates.IsSalesperson', 'UserPreferences': 'updates.UserPreferences', 'PhoneNumber': 'updates.PhoneNumber', 'FaxNumber': 'updates.FaxNumber', 'EmailAddress': 'updates.EmailAddress', 'Photo': 'updates.Photo', 'CustomFields': 'updates.CustomFields', 'OtherLanguages': 'updates.OtherLanguages', 'LastEditedBy': 'updates.LastEditedBy', 'ValidFrom': 'updates.ValidFrom', 'ValidTo': 'updates.ValidTo', 'RecordLoadDate': 'updates.RecordLoadDate'}


In [14]:
deltaTable = DeltaTable.forPath(spark, f'{target_data_path}')

merge = deltaTable.alias('original') \
    .merge(dfDataChanged.alias('updates'), 'original.HashedPKColumn = updates.MergeKey') \
    .whenMatchedUpdate(
        condition="updates.HashedNonKeyColumns <> original.HashedNonKeyColumns and original.IsCurrent = 1  ",
        set={
            "IsCurrent": lit(0),
            "RecordEndDate": col('updates.RecordStartDate')
        }) \
    .whenMatchedUpdate(
        condition="original.IsCurrent = 1 AND original.IsDeleted = 0 AND updates.Action = 'D'",
        set={
                "IsDeleted": lit(1)
        }) \
    .whenNotMatchedInsert(
        values={**columns_to_insert,
                "HashedPKColumn": col("updates.HashedPKColumn"),
                "HashedNonKeyColumns": col("updates.HashedNonKeyColumns"),
                "IsCurrent": lit(1),
                "RecordStartDate": current_timestamp(),
                "RecordModifiedDate": current_timestamp(),
                "RecordEndDate": lit('9999-12-31').cast('timestamp'),
                "IsDeleted": lit(0)})

# Execute the merge operation
merge.execute()

StatementMeta(, 6b0e549f-d44f-4d95-a0d3-c56f16f829b0, 16, Finished, Available)

## Exit notebook

In [15]:
TotalRuntime = str((datetime.datetime.now() - start_audit_time)) 

# Your data
result_data = {
    "CopyOutput":{
        "Total Runtime": TotalRuntime,
        "TargetSchema": target_schema,
        "TargetName" : target_name
    }
    }


mssparkutils.notebook.exit(result_data)

StatementMeta(, 6b0e549f-d44f-4d95-a0d3-c56f16f829b0, 17, Finished, Available)

ExitValue: {'CopyOutput': {'Total Runtime': '0:00:27.290202', 'TargetSchema': 'Application', 'TargetName': 'People'}}